In [1]:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the first GPU


True
1
NVIDIA GeForce GTX 1660 SUPER


In [3]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224, or other preferred size
    transforms.ToTensor(),  # Convert to tensor
])

# Load the datasets
train_data = datasets.ImageFolder(root='C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\training-images', transform=transform)

# DataLoader to iterate through the dataset
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Freeze the convolutional base if you want to fine-tune only the custom layers
for param in model.parameters():
    param.requires_grad = False

# Modify the loss function
criterion = nn.BCEWithLogitsLoss()

# Ensure your model's output layer produces raw logits (no Sigmoid in the forward pass)
class CustomClassificationHead(nn.Module):
    def __init__(self, in_features):
        super(CustomClassificationHead, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 1)  # Single output unit (logits)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Replace the ResNet fc layer with the custom classification head
model.fc = CustomClassificationHead(model.fc.in_features)

# Send the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


C:\Users\redfr\elevenvenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\redfr\elevenvenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy with logits
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Reshape labels to match the output shape and ensure they're of type float
        labels = labels.view(-1, 1).float()  # Reshape to (batch_size, 1) and convert to float
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print average loss after each epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.37230116995030194


In [ ]:
# Load the test dataset
test_data = datasets.ImageFolder(root='C:\\Users\\redfr\\Downloads\\archive\\for-2sec\\for-2seconds\\testing-images', transform=transform)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
# Set the model to evaluation mode
model.eval()

# Initialize variables to track accuracy
correct = 0
total = 0

# Disable gradient computation for evaluation
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Get predictions
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the class index with the highest score

        # Update total and correct counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate and print accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")